In [1]:
print("hello, world!")

hello, world!


In [1]:
# system packages 
import os
import json

# internal packages 

# external packages
from openai import OpenAI
from dotenv import load_dotenv
from graphql import build_schema, parse
from jinja2 import Environment, FileSystemLoader

load_dotenv()
open_ai_api_key = os.getenv("OPENAI_API_KEY")

## GraphQL Schema Parsing

In [14]:

with open('../assets/schemas/opensea/opensea_original_schema.graphql', 'r') as schema_file:
    schema_str = schema_file.read()

parsed = parse(schema_str)
print(parsed.to_dict())

{'kind': 'document', 'definitions': [{'kind': 'enum_type_definition', 'description': None, 'name': {'kind': 'name', 'value': 'Network'}, 'directives': [], 'values': [{'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'ARBITRUM_ONE'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'ARWEAVE_MAINNET'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'AURORA'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'AVALANCHE'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'BOBA'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'BSC'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'CELO'}, 'directiv

## OpenAI Directly

In [3]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

def gpt_chat_completion(client, message, model="gpt-4o"):
    return client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": message,
        }
    ],
    model=model,
)

rep = gpt_chat_completion(client, "Hello! How are you?")
print(rep.choices[0].message.content)

Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?


## Initial Test Set 

### Prompts 

- [ ] One entity quality vs. another entity quality 
- [ ] One column quality vs. another column quality 
- [ ] One schema quality vs. another schema quality 

### Demonstrations 

- [ ] A high quality entity (description)
- [ ] A high quality column (description)
- [ ] A high quality schema (description)

### Quality Categories

- [ ] 

In [4]:
with open("tests/assets/entity_comparison_assets.json", "r") as f:
    entity_comparison_assets = json.load(f)

gold_entity_comparison = "".join(entity_comparison_assets["gold"]["prompt"])
four_entity_comparison = "".join(entity_comparison_assets["four"]["prompt"])
three_entity_comparison = "".join(entity_comparison_assets["three"]["prompt"])
two_entity_comparison = "".join(entity_comparison_assets["two"]["prompt"])
one_entity_comparison = "".join(entity_comparison_assets["one"]["prompt"])

In [5]:
# Function to reload the template
def reload_template(template_path="../assets/prompts", template_name="entity_comparison_prompt.txt"):
    env = Environment(loader=FileSystemLoader(template_path))
    env.cache.clear()
    entity_comparison_prompt_template = env.get_template(template_name)
    return entity_comparison_prompt_template

def parse_response(response): 
    response = response.choices[0].message.content
    response = response.strip('```json').strip('```').strip()
    response = json.loads(response)
    return response

example_prompt = reload_template()
example_prompt_filled = example_prompt.render({"entity_pred": "<example prediction goes here>", "entity_gold": "<example gold goes here>"})

entity_comparison_prompt_template = reload_template()
gold_output = entity_comparison_prompt_template.render({"entity_pred": {gold_entity_comparison}, "entity_gold": {gold_entity_comparison}})
four_output = entity_comparison_prompt_template.render({"entity_pred": {four_entity_comparison}, "entity_gold": {gold_entity_comparison}})
three_output = entity_comparison_prompt_template.render({"entity_pred": {three_entity_comparison}, "entity_gold": {gold_entity_comparison}})
two_output = entity_comparison_prompt_template.render({"entity_pred": {two_entity_comparison}, "entity_gold": {gold_entity_comparison}})
one_output = entity_comparison_prompt_template.render({"entity_pred": {one_entity_comparison}, "entity_gold": {gold_entity_comparison}})

print("Expecting output: 3 \n")
gold_comparison = parse_response(gpt_chat_completion(client, gold_output))
print(gold_comparison)

print("\n Expecting output: 4 \n")
four_comparison = parse_response(gpt_chat_completion(client, four_output))
print(four_comparison)

print("\n Expecting output: 3 \n")
three_comparison = parse_response(gpt_chat_completion(client, three_output))
print(three_comparison)

print("\n Expecting output: 2 \n")
two_comparison = parse_response(gpt_chat_completion(client, two_output))
print(two_comparison)

print("\n Expecting output: 1 \n")
one_comparison = parse_response(gpt_chat_completion(client, one_output))
print(one_comparison)  

Expecting output: 3 

{'reasoning': "To evaluate the documentation provided, I will compare it against the GOLD documentation and the criteria provided. The task requires determining whether the evaluated documentation matches or exceeds the quality of the GOLD documentation, based on criteria of correctness, clarity, and descriptiveness.\n\nFirst, we need to examine the documentation submitted for evaluation:\n\n- The 'id' field contains a clear and concise description, matching the GOLD example.\n- Similarly, the 'marketplace' field is well-described, stating which marketplace the snapshot belongs to, consistent with the GOLD documentation.\n- The 'blockNumber' and 'timestamp' fields are described accurately, specifying where and when the snapshot is taken, aligning with the GOLD standard.\n- The 'collectionCount', 'cumulativeTradeVolumeETH', 'marketplaceRevenueETH', 'creatorRevenueETH', 'totalRevenueETH', 'tradeCount', 'cumulativeUniqueTraders', 'dailyActiveTraders', 'dailyTradedCol

In [6]:
entity_comparison_prompt_template = reload_template(
    template_path="../assets/prompts",
    template_name="entity_comparison_revision.txt",
)

entity_comparison_prompt_modification = entity_comparison_prompt_template.render(
    {
        "original_prompt": {example_prompt_filled}, 

        "four_result_correct": {"True" if four_comparison["correctness"] == 4 else "False"},
        "four_result": {four_comparison["reasoning"]},
        
        "three_result_correct": {"True" if three_comparison["correctness"] == 3 else "False"},
        "three_result": {three_comparison["reasoning"]},
        
        "two_result_correct": {"True" if two_comparison["correctness"] == 2 else "False"},
        "two_result": {two_comparison["reasoning"]},
        
        "one_result_correct": {"True" if one_comparison["correctness"] == 1 else "False"},
        "one_result": {one_comparison["reasoning"]},
    }
)

prompt_revision = gpt_chat_completion(client, entity_comparison_prompt_modification)

In [7]:
file_path = "../assets/prompts/entity_comparison_prompt_revised.txt"

prompt_revision = parse_response(prompt_revision)
revised_prompt = prompt_revision["modified_prompt"]

# Replace the placeholder with Jinja syntax
revised_prompt = revised_prompt.replace(r"{entity_pred}", r"{{ entity_pred }}")
revised_prompt = revised_prompt.replace(r"{entity_gold}", r"{{ entity_gold }}")

with open(file_path, "w") as file:
    file.write(revised_prompt)